1.导入需要的库

In [ ]:
#数据处理和科学计算库
import numpy as np
import pandas as pd

#可视化库
import matplotlib.pyplot as plt
import seaborn as sns

#机器学习库
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

#统计分析库
import statsmodels.api as sm
from scipy import stats

#设置可视化风格
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签（字体：微软雅黑）
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

2.数据生成与分析

In [ ]:
np.random.seed(42)

#生成100个学生的学习时间（小时）
study_hours = np.random.uniform(1, 10, 100).reshape(-1, 1)

#生成对应的考试成绩（分数），假设基础分为30，每小时的学习可以增加7分，并加入一些随机噪声
exam_scores = 30 + 7 * study_hours + np.random.normal(0, 5, 100).reshape(-1, 1)

#将数据转换为Pandas DataFrame,方便后续处理
data = pd.DataFrame({'Study_Hours': study_hours.flatten(), 'Exam_Scores': exam_scores.flatten()})

In [ ]:
print("数据前5行：")
print(data.head())

In [ ]:
print("数据的基本信息：")
print(data.describe())

#缺失值检查 
##均值填充
##删除缺失值

#异常值分析

2.数据可视化：散点图

In [ ]:
plt.figure(figsize = (10, 6)) #创建画布
sns.scatterplot(x = 'Study_Hours', y = 'Exam_Scores',data = data, alpha = 0.7) #散点图绘制(alpha代表点的大小)
plt.title('学习时间与考试成绩的关系', fontsize = 15)
plt.xlabel('学习时间（小时）', fontsize = 12)
plt.ylabel('考试成绩（分）', fontsize = 12)
plt.show()


In [ ]:
#计算相关系数
correlation = data['Study_Hours'].corr(data['Exam_Scores'])
print(f"学习时间与考试成绩的相关系数为：{correlation:.4f}")

3.划分训练集和测试集

In [ ]:
#定义自变量X和因变量Y
X = data[['Study_Hours']] #自变量必须是二维数组！
Y = data['Exam_Scores']

#按照80%和20%的比例划分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
print(f"训练集大小：{X_train.shape[0]}")
print(f"测试集大小：{X_test.shape[0]}")

4.模型训练

In [ ]:
#1.创建线性回归模型实例
model = LinearRegression()

#2.使用训练数据来训练模型
model.fit(X_train, Y_train)

#3.获取模型的参数
intercept = model.intercept_ #截距（beta_0）
slope = model.coef_[0] #斜率(beta_1)

print("模型训练完成")
print(f"截距(beta_0)：{intercept:.4f}")
print(f"斜率(beta_1)：{slope:.4f}")

print('回归方程为：')
print(f'Exam_Scores = {intercept:.2f} + {slope:.2f} * Study_Hours')



5.模型预测与评估

In [ ]:
#在测试集上进行预测
Y_pred = model.predict(X_test)

#将预测结果和实际结果放在一个DataFrame里方便比较
predictions_df = pd.DataFrame({'Actual': Y_test, 'Predicted': y_pred})
print("测试集上的实际结果vs预测结果")
print(predictions_df.head())

5.1计算评估指标

In [ ]:
#计算R²(决定系数)
r2 = r2_score(Y_test, Y_pred)
#计算MSE(均方误差)
mse = mean_squared_error(Y_test, Y_pred)
#计算RMSE（均方根误差）
rmse = np.sqrt(mse)
#计算MAE（平均绝对误差）
mae = mean_absolute_error(Y_test, Y_pred)

print("--- 模型在测试集上计算出的评估指标 ---")
print(f'R-squared (R²):{r2:.4f}')
print(f'Mean Squared Error (MSE):{mse:.4f}')
print(f'Root Mean Squared Error (RMSE):{rmse:.4f}')
print(f'Mean Absolute Error (MAE):{mae:.4f}')


6.结果可视化

In [ ]:
plt.figure(figsize = (10, 6))

#绘制数据散点图（测试集）
plt.scatter(X_test, Y_test, color = 'blue', label = '实际成绩', alpha = 0.7)

#绘制回归线
plt.plot(X_test, Y_pred, color = 'red', linewidth = 2, label = '回归线')

plt.title('线性回归拟合结果（测试集）', fontsize = 15)
plt.xlabel('学习时间（小时）', fontsize = 12)
plt.ylabel('考试成绩（分）', fontsize = 12)
plt.legend()
plt.show()

6.1残差图分析

In [ ]:
#计算残差
residuals = Y_test - Y_pred

plt.figure(figsize = (10, 6))
sns.scatterplot(x = Y_pred, y = residuals, alpha = 0.7)
plt.axhline(y = 0, color = 'red', linestyle = '--')
plt.title('残差图', fontsize = 15)
plt.xlabel('预测值', fontsize = 12)
plt.ylabel('残差', fontsize = 12)
plt.show()

7.使用模型进行新预测

In [ ]:
#假设一个新学生学习了6.5个小时，用模型来预测他的成绩
new_study_hours = np.array([[6.5]])

#使用模型进行预测
predicted_score = model.predict(new_study_hours)

print(f'预测：学习6.5小时的学生的考试成绩约为{predicted_score[0]:.2f}分') 

8.封装好的函数，以后可以直接调用

In [ ]:
def run_linear_regression_analysis(data, feature_col, target_col):
    """
    执行一元线性回归分析的完整流程模板。
    
    参数:
    data (pd.DataFrame): 包含数据的DataFrame。
    feature_col (str): 自变量列名。
    target_col (str): 因变量列名。
    
    返回:
    dict: 包含模型、评估指标和预测结果的字典。
    """
    # 1. 数据准备和划分
    X = data[[feature_col]]
    y = data[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 2. 模型训练
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 3. 预测与评估
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # 4. 打印结果
    print(f'--- 分析报告: {feature_col} vs {target_col} ---')
    print(f'回归方程: {target_col} = {model.intercept_:.2f} + {model.coef_[0]:.2f} * {feature_col}')
    print(f'R-squared (R²): {r2:.4f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
    
    # 5. 可视化
    plt.figure(figsize=(10, 6))
    plt.scatter(X_test, y_test, color='blue', label='实际值')
    plt.plot(X_test, y_pred, color='red', linewidth=2, label='预测回归线')
    plt.title(f'{feature_col} 与 {target_col} 的回归分析')
    plt.xlabel(feature_col)
    plt.ylabel(target_col)
    plt.legend()
    plt.show()
    
    return {
        'model': model,
        'r2': r2,
        'rmse': rmse,
        'predictions_df': pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    }

# 使用模板函数
results = run_linear_regression_analysis(data, 'Study_Hours', 'Exam_Scores')